In [ ]:
!pip install Bio

In [76]:
from Bio import SeqIO
from Bio.SeqUtils import GC

from Bio.Data.CodonTable import unambiguous_dna_by_id
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from random import choice
from random import shuffle

# https://biopython.org/wiki/Degenerated_Codons
def altcodons(codon, table = 1):
    """List codons that code for the same aminonacid / are also stop.

    @param codon
    @table code table id
    @return list of codons

    """
    tab = unambiguous_dna_by_id[table]

    if codon in tab.stop_codons:
        return tab.stop_codons

    try:
        aa = tab.forward_table[codon]
    except:
        return []

    return [
        k
        for (k, v) in tab.forward_table.items()
        if v == aa and k[0] == codon[0] and k[1] == codon[1]
    ]

record = SeqIO.read("cds.fna", "fasta")
record = record.seq

# Pregunta 3


Lea el siguiente ejemplo de código provisto en el “Cookbook” de Biopython:
http://biopython.org/wiki/Degenerated_Codons. Incluye una función, degenerated_subseq, que toma una secuencia codificadora (es decir, la parte de un gen que parte con AUG y termina con un codón de Stop),
e imprime una subsecuencia que se salta todos los codones que pertenezcan a aminoácidos con menos de
'x' codones. Por ejemplo, si se pidió x=3, entonces un codón UUC no aparecería en el output, pues su
aminoácido es codificado sólo por 2 codones (UUC y UUU); en cambio, un codón UCA sí aparecería,
pues su aminoácido es codificado por 6 codones distintos. Nos interesa hacer algo diferente, pero este
código y sus funciones auxiliares debieran resultar iluminadores para poder hacer lo que se pide, sin que
tengan que revisar el resto de la documentación de Biopython.


a) Escriba una función que tome una secuencia codificadora y la aleatorice, reemplazando cada
codón por un codón al azar escogido de entre los que codifican al mismo aminoácido. Por
ejemplo, donde hay un UUC, pondremos ya sea UUC o bien UUU, con 1/2 de probabilidad para
cada uno; donde haya un UCA, pondremos cualquiera de los 6 codones que codifican serina, con
probabilidad 1/6 para cada uno.

In [68]:
def sequenceRandomizer(seq):
    data = ""
    for i in range(0, len(seq), 3):
        codon = seq[i : i + 3] 
        altCodon = choice( altcodons(codon) )
        data += altCodon
    return data

b) Escriba una función que tome una secuencia codificadora y la reemplace por otra que codifica
exactamente la misma proteína, pero que tiene el mayor porcentaje de G y C posible.

In [69]:
def getMaxGCsequence(seq):
    data = ""
    for i in range(0, len(seq), 3):
        codon = seq[i : i + 3]

        GCValues = []
        alternativeCodons = altcodons(codon) 
        for altCodon in alternativeCodons:
            GCValues.append( GC(altCodon) )
        
        options = []
        maxGCValue = max(GCValues)
        for index, value in enumerate(GCValues):
            if value == maxGCValue:
                options.append( alternativeCodons[index] )
        
        maxGC_codon = choice( options )
        data += maxGC_codon
    return data

c) Aplique sus funciones al archivo cds.fna, y guarde los resultados en archivo cds_random.fna y
cds_maxGC.fna (también en formato FASTA).

In [71]:
gcValue0 = round( GC(record), 2)

sequence1 = sequenceRandomizer(record)
gcValue1 = round( GC(sequence1), 2)
newRecord1 = SeqRecord(
    Seq( sequence1 ),
    id = "Secuencia Random",
    description = f"| G+C content {gcValue1}",
)

sequence2 = getMaxGCsequence(record)
gcValue2 = round( GC(sequence2), 2)
newRecord2 = SeqRecord(
    Seq( sequence2 ),
    id = "Secuencia con Max G+C",
    description = f"| G+C content {gcValue2}",
)

with open("cds_random.fna", "w") as file:
    SeqIO.write(newRecord1, file, "fasta")

with open("cds_maxGC.fna", "w") as file:
    SeqIO.write(newRecord2, file, "fasta")

print(f"G+C content de cds.fna: {gcValue0}")
print(f"G+C content de cds_random: {gcValue1}")
print(f"G+C content de cds_maxGC: {gcValue2}")

G+C content de cds.fna: 41.22
G+C content de cds_random: 46.16
G+C content de cds_maxGC: 62.57


d) Sus funciones en a) y en b) están realizando traducción inversa de proteínas, pues están
creando un gen a partir (en el fondo) de una secuencia de proteína dada. Sugiera una forma de
realizar esta traducción inversa de modo que el gen resultante tenga (aproximadamente) un nivel
de %GC ingresado como argumento. [No la programe, sólo describa su estrategia.]

Dada la secuencia de la proteína, se podría construir una secuencie que codifique la mísma proteína pero maximizando el contenido de G-C (análogo a lo desarrollado en b), luego iterar codón por codón y re-codificarlo, esto haría que el contenido de G-C disminuya o se mantenga igual para cada iteración. Repetir el procedimiento hasta alcanzar el umbral deseado. El procedimiento podría realizarse de manera incremental: i, i+1, i+2, etc. De manera secuencial con salto: i, i+k, i+2k, etc. O de manera aleatoria 

# Pregunta 4

Lea con detención la sección 20.1.13 del tutorial de Biopython,
http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec384. Incluye dos versiones de código para listar
todos los ORFs de una secuencia leída desde un archivo en formato FASTA.

a) Modifique el final del código (cualquiera de las dos versiones) para que sólo informe los tres
    ORFs con porcentaje de G+C más alto, y los tres con porcentaje de G+C más bajo

b) Aplique su función al archivo NC_005816.fna

In [8]:
record = SeqIO.read("NC_005816.fna", "fasta")
table = 11
min_pro_len = 100

In [16]:
array = []

for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
    for frame in range(3):
        length = 3 * ((len(record) - frame) // 3)  # Multiple of three
        for pro in nuc[frame : frame + length].translate(table).split("*"):
            if len(pro) >= min_pro_len:
                print(
                    f"{pro[:30]}...{pro[-3:]} - length {len(pro)}, strand {strand}, frame {frame}"
                )
                array.append((GC(pro), pro))

array.sort()
array.reverse()

print()

if len(array) > 3:
    array = array[:3]

for valor, seq in array:
    print(
        f"{seq[:30]}...{seq[-3:]} - length {len(seq)}, G+C content {round(valor, 2)}"
    )

GCLMKKSSIVATIITILSGSANAASSQLIP...YRF - length 315, strand 1, frame 0
KSGELRQTPPASSTLHLRLILQRSGVMMEL...NPE - length 285, strand 1, frame 1
GLNCSFFSICNWKFIDYINRLFQIIYLCKN...YYH - length 176, strand 1, frame 1
VKKILYIKALFLCTVIKLRRFIFSVNNMKF...DLP - length 165, strand 1, frame 1
NQIQGVICSPDSGEFMVTFETVMEIKILHK...GVA - length 355, strand 1, frame 2
RRKEHVSKKRRPQKRPRRRRFFHRLRPPDE...PTR - length 128, strand 1, frame 2
TGKQNSCQMSAIWQLRQNTATKTRQNRARI...AIK - length 100, strand 1, frame 2
QGSGYAFPHASILSGIAMSHFYFLVLHAVK...CSD - length 114, strand -1, frame 0
IYSTSEHTGEQVMRTLDEVIASRSPESQTR...FHV - length 111, strand -1, frame 0
WGKLQVIGLSMWMVLFSQRFDDWLNEQEDA...ESK - length 125, strand -1, frame 1
RGIFMSDTMVVNGSGGVPAFLFSGSTLSSY...LLK - length 361, strand -1, frame 1
WDVKTVTGVLHHPFHLTFSLCPEGATQSGR...VKR - length 111, strand -1, frame 1
LSHTVTDFTDQMAQVGLCQCVNVFLDEVTG...KAA - length 107, strand -1, frame 2
RALTGLSAPGIRSQTSCDRLRELRYVPVSL...PLQ - length 119, strand -1, frame 2

RALTGLSAPGIRSQTSCDRLRELRYV

# Pregunta 5

El archivo plasmido.fna contiene el genoma de un cierto plásmido de Escherichia coli.

a) Escriba un pequeño código en Python que lea un archivo de este tipo y genere un archivo
similar, pero en que los nucleótidos hayan sido reordenados al azar (manteniendo la cantidad de
cada tipo). Se sugiere usar la función shuffle del módulo random.

In [85]:
def randomizeSequence(nomarch):
    record = SeqIO.read(nomarch, "fasta")
    secuencia = list(record.seq)    # se pasa a lista
    shuffle(secuencia)              # se reordena
    secuencia = "".join(secuencia)  # se pasa a string
    return secuencia



b) Use su función para generar una versión aleatorizada del genoma del plásmido.


In [88]:
randomSeq = randomizeSequence("plasmido.fna")
randomSeq[:10]  # se aprecia que es diferente al contenido del archivo

'AGCCCACCAT'

In [89]:
randomSeq

'AGCCCACCATCCCCGACATCCCCTCGGATCATATGATGGACCCGCCACTTCACTATGCTTGTGGTTTTTAAGGATGCGAACGCGAGGTCGCCGGCTTGGCTCATCGGGATAATATTTACGGCATTGCGATCTTAGCGCAAAACGTCGGCCCGGTTGCGACTATGTGCTGGGCACTTCGCGAGGCGCTTGTACGAGCACTCAGTCCAACTGATTATTACACAAGGGGAAGAGGTACTGAGATCGACGGGCGACGCTTTCCATGCTTTCGTTTCTTGAAATATGGACAGCTGCAGCGTTAGTGAGGCCCTCTGAAGTCCAATAAGTAGAACGGGAGGCCTCCGACCACAGATTCTCAACCTCGCCCTTCGCAGGCGATATGCGGAACACCCGTTCAATGACCCGGCCCATAAGAACCTGCAGCACTGCAAAAGGGACAGCCCCACCCATCGCACCACCACCTCCGATCACCCAGCCGAGTTATAAGACGGGGACCGCCCAGCGGCATGCTTTGTAACACAGCGCCTACTCATGACCACAATAGGCATCGCCAGCTGTGATCTCCTTACTCCCGAGCATACGTAACCCTTATTCTCAGCTAAACCTCCTCGGGAATAGCTCTCAGTCCCGGGAATTGATTATTCTGCCTTTACACACGCCGTTAAGCAGAAGCCATTCACATGCGTCAACCATGTCGTAACATGGTACTGACCAACTCCCGGCGATACATTGAAGCTGGGATCTCCCGGGAAAGGCAACAAACCATGCGTCCAACGTTGCCCAGCCGGTGCGTTAGACTTCCCCTATCGTTACCATTTGCTCACGATTTGGCTATGGCCAAGTATATGATTTCCCTCTATTAGTGTCAGGTGCTCTTTTTGCTGCGGCAACATGGACAACTGGCGCTCCTCAGCAAACCCGCACCGTATTATGCGATTTGACTCAATTGTAGCGACTGGAGTCCAACAATCTTCCCCATCCACCGCCATGATACGAGGGATT

Se jecutó la query con esta secuencia:

AGCCCACCATCCCCGACATCCCCTCGGATCATATGATGGACCCGCCACTTCACTATGCTTGTGGTTTTTAAGGATGCGAACGCGAGGTCGCCGGCTTGGCTCATCGGGATAATATTTACGGCATTGCGATCTTAGCGCAAAACGTCGGCCCGGTTGCGACTATGTGCTGGGCACTTCGCGAGGCGCTTGTACGAGCACTCAGTCCAACTGATTATTACACAAGGGGAAGAGGTACTGAGATCGACGGGCGACGCTTTCCATGCTTTCGTTTCTTGAAATATGGACAGCTGCAGCGTTAGTGAGGCCCTCTGAAGTCCAATAAGTAGAACGGGAGGCCTCCGACCACAGATTCTCAACCTCGCCCTTCGCAGGCGATATGCGGAACACCCGTTCAATGACCCGGCCCATAAGAACCTGCAGCACTGCAAAAGGGACAGCCCCACCCATCGCACCACCACCTCCGATCACCCAGCCGAGTTATAAGACGGGGACCGCCCAGCGGCATGCTTTGTAACACAGCGCCTACTCATGACCACAATAGGCATCGCCAGCTGTGATCTCCTTACTCCCGAGCATACGTAACCCTTATTCTCAGCTAAACCTCCTCGGGAATAGCTCTCAGTCCCGGGAATTGATTATTCTGCCTTTACACACGCCGTTAAGCAGAAGCCATTCACATGCGTCAACCATGTCGTAACATGGTACTGACCAACTCCCGGCGATACATTGAAGCTGGGATCTCCCGGGAAAGGCAACAAACCATGCGTCCAACGTTGCCCAGCCGGTGCGTTAGACTTCCCCTATCGTTACCATTTGCTCACGATTTGGCTATGGCCAAGTATATGATTTCCCTCTATTAGTGTCAGGTGCTCTTTTTGCTGCGGCAACATGGACAACTGGCGCTCCTCAGCAAACCCGCACCGTATTATGCGATTTGACTCAATTGTAGCGACTGGAGTCCAACAATCTTCCCCATCCACCGCCATGATACGAGGGATTTCGGAGCAGGTGCACATATGGGGTCCGCTATCACTGGTGGTACCTTTACCCCGTTGTATTATAGGAAATAGCAGATATTAGCTCGCAGTGCCGATGCCACCTTCGATGCGGCCAATGCCGGTCAAAGGTCACGGGAGCGTTCGGGGCCCCGCTAGGCGGCCCCTGCATTAGAAACTCCTGCCCGCCCCTTGCGACATTCCCGGAACAACGTAAACCCGCTTGACATCTAGTCTGATTCGGGCTGGCAGGCCATTCGTCGTGACGATACCCTGACGATCTGCGGTTGGCAGGGCTTAGGCCCCGTACCCGTGGCCAGTGGTCGGTCGCGTTTCTCACAGGCCCTCTGCGACCGTGGGCACTGGGACCCCCCCTCCACATTTATCGCATCAACGTACTGCACAAGGATGCATACACTCATTTATCCTCGCGATCGGCCTACCCCCAGTACAGAGCGGCGAAGACAGGCCATCGAAACTCTCAAATCGTCACACCACTCCTGATACGGGTACTTACAGCGATAGACTGCCATCAAGGGAAAAGAGTGCGTAGATGTCGCAGTCAGGTCACCCCTTCACTTTTAGCATTGAAGTTCGCCCAGCAAGTCGGGTCAAGAAACCCAGTCCCTTGACGACTCTTTACAGACATGCGCCTAAACCTGCAGTCCCTCAAACTCTTCAAGCACGCCCCCCCCCCTACAGTTCCCCTCCGGGGAAACGACTTGCCCGCATGCATTCCCCTGTGGTCATGAAGTAGAGCCATGCAGAAGGAAGCTAGACCCTGACGCCTATGTATCTCCCGTCGAGCGATAGAGATTTGTTCCACCGTATCTTCGATGGATCACCCGCGGAACCTCTCTGCCACAAAGACCACGATAGCGCTAGCGACGCGTCTTCCGCGTGACCCACGGCCAAACTTTAGAAGCGCTCTTCCACAGACCACACACGTCCGAAACAGGCGATCCTAGTAATATAGCTAAGGGCCTGCGCGGCGCGCCGCCGGCCGGCCAATGTTTTCGGGGTTACCTCATCTGTATAACTCTAGATCCTAGTTGGGAGCACCGCGACTGTTGGTCTTGGGGGCCGCCGAAGGCCTCAACTTAGCTACATTTGAAAGTGCTAGATGGGGCAATTCCGCCCACATCCGTCTCGGCCTGATTGCTCAGGTCGTCGATCAGGTTCCCACAAAGCATACGCGAACTGATCTGGTGTCTATCACAGTTGACTACCGCTCAAAGATAATATCCTGGGACGTGGTCCTAGACGAACTCTGCTGCGAGGAGTGAAGTATATATGCCACGTACGGCGCACAACATTCATTCAGAAGCCCGCCGAGCGCTTGACCTCTTCTCCTGAAGCCGTACCAGTCGGTCCGTACCATCGCCGCTAGCGTCAGAGTAGCTACCACCCCATCGAAACGCTTCTAGAAGGACACGACCGAACGCGGGCGCTAGTGATTGCATACGGTGTCGCTCCTCGTAAACCTGACTTTCGAGGCGTTAGACAGAATGCTCACGCACTCTCTGTTTCAACTATCTCTGCGTGGAGGCGCTCTACAGACGTGTCAATCAGATTCGAGGCATCACCGTGGCAAAGTAATGCGCTTAATAACACTAAGCCTTACTATTATAGGAGAGACATAAGAGGCTCCTTGTCTCTTGTGGTTGTCTATACGCTCTTAACCTCCCCATGAGAGCATTTCCGACTCCCCCGTAAGATGTGCAGGGTGATACAGTCCATCCGACCAAAAGGGGCTCGGGCGAAGAATAGCACACTCAATTGGAACCCTGGCTCAACGCATTACCAGACCGACGAGAAAGCGGCTAGCGTAAAAACCTGCGATAGTAATGCTAACTAGCTTACAAGTCCGCTAACGGCTCGAGGCATTCCTGTAAGCCCCAATCGTACCAAGTTCCAGGTCTGCCCGTGTTCGACGCAGTCTTTTGTCACTCACACGCACAAGCCGGGCTTATATCTCTGAATCAACCGGTAACCCGGTATCTCATTGGCTTAAGATGCAGCCACACCCCACCAGTCGTCTCATCTGTGGTACGTTCATAACACTATCATGCTTCGCTCTGTAGGCCCGCAACCGATTGCTCAGCTTACGACCCGAGCCGCTACCCTACCCTCCGTCTACTGTCTGATCCCTCGGCCGTGCTCCCCAACACATTCTAGAATCGCTTTGGTCGCGTCACGCCCTACAGTCGACATCACCTCTCACAGACACTCACGCATGAACATCAATTGAGTTGCTTGATGAGCGGAAATGCGCATCCGAATATTGATACCTCCCTCCCTATATATAGCCGGATCGTTCAGTGCACTATTGCTCTGAGCGACACCCACTGTGGATGCACGCCCCACCCTGCCCACTGCTCTCTTAGCCCACCGTGAGTCCCTATCATGTTCCAGTAGCGGAGCCGGCATATGGCATAGAGATAACCAAAGAGACCCTTCGTCCTGTATTTTATACTCTTATCGCGTTCCGGAGCCGTGGAGGTTTAGCCAGAACGGCTCGAATCGTTACCCGGGCGGTCCTGACCGTACAAATATGCTATAATCCTGTGGTTACCCCAAGATTTGCCAGCGTTGTGCCGACAAGAACAGTCGGTTGCAGCTTGCGCTACATCAAGCGTACGTTTGGGACCACTATAGGCACTAGCACAGCTGCAACTACGAAATTGTTTCCCGCTTAACACCGATGGGTTAACGCGGGTACCGATTAGCCCCAAATCTTTTGGGAACCCGAAGCCTTTCGGACCAGCGCAGCCCGGGGCTTCCAGCTACCCCCCCCTGCCAGGTCCACCAAAGATGCCATGGAGCATAGACCCACCTCGACTGTGCTCCTAAGAACTGATACCGTTGATTCGTCGTTCCACTATTCTTGGCTACACTTCGGCAGATGTGGTCGTCGCGGAAGATCTGTTTACGCGAAAGTAGCTTAGGGTTCCGTTTCCCTGAAGTGCTGGCAAAGCGTTGTATTCATGGGAGCTCGCCAATGGAGACGTGCTCCGTACCATCCCTCCTGTTTAATCGTCATGTAGGCGCTTGTGAAGTAATCCTCAACGCAACCGTAGGAGCACGTTAACAGAGATGCCCTGATCCCCTTTTTTAGCTACCCTCGTACTTAAAGCAGGCAGAGATTGGTATAAATTAGCTGCCGCGACCTGAAACCACGATAGTCTCTAGTTGTGGTGGGTTGAGCCATACTGGGTGTATAACAACCATCTCCTCGGATTAACACACCCACGGACTTGCTGCCTCAGCAAAAATGTCATGACGTAAGACGCAAGTGTCCTAAATAAGCTCTAACCCGCTGGCCCGAGGACGCCCCTAGCTAGTGAACAACGCCAAGCGTTTGGGGCTCAGTATCTAATTCACTGGAGGCCCGGCCTTATAGGCCCACCAGGTCCCGCTTCGATTTATGTCCTCCCAGAAGTCCCGAAATAATCCATTTGTACAACCCAACTAATAAACGGTTCTCTTCCATATAAGGGGGGTGTTTGAACCAGCCGAGTGATAGCTTCTGGAGGCGCATACGTGCTGCCCTCCCCAGTTATATACCTCCGCCTCTATCTCTCGTTGTATGGACTGATCTGACCCCCGTGCTGGATAAGCCTGCGTCCCTCCGACTCCCGCAAGTCCACCCCTGCGAATTCTTGATGACCTGTTCACGTACCCACCCTATTGCATACTTACAGTGATCACACGACATGCAACAGCTCCGGCATACTTACCGCTAGCACTGCCAGGCAGTTTTCGTCGAAACAGAGTAATTTCGTGATCCAAGCAACGCGGTTGCACCCGGACGAGGCTCTACCGTGAGTCGGCCACTGTCTCTACCCATTCTCAAACGTGGTCGGGCACAATACCCAATCCCAAGCACCGCAAGTGGATCGAATTATTGTTCAAGCTAGCTTTTCGCTCATCTTCAGGCAACCAGTTCTATTCCGAATGAATTCGGCGCACATGTAGCATCTCGGAGCCCCACGCGACCCGGTCAGGTGTCAAACGCCTGCCCTCGGGCTAACGTGTGGATGTGGACCACACGATCATAGGTGTCACATCGCCCACATCGTTACTCACGGCATTCACGCCAGTTAATGACCTACTCGGCTAAGCTACGCAACTGGTCAAACACACAAATCGTCCATTTCCAGAGCGCTCAAGTGCTTTGATTTTCTACCTACCAATGCATCTCAAATGCACCTAACCAGCTCGGCTTGGACAGCTTCGGGGTTACGCAGCTTCAACCACTGTCGAACTACCAACATCACTGGATCGGTTGGAGTCGCACCTAGCCGGTCGCAGCTCTTGACGAAACTAAAAGACCTTCTGCCCCGCCTCTCTCCACAAACTAGTCATGGTCCGGCCTTGCCAGCTGTTAGCACGCGCGCCTGATCTCCAACTCCGTCGAGCAAGTGTGTCGATCGCGTCTGCTCGTGATTCCGAATACGGACTCCATACCTCTGTTCCAAATCCAGTGGGGCTGCGCATCGTGGTGGCTTACCAGGGGTGAGCATAAAACTGAACGGACGCTTGCCCTGTCGCTGCCGTCTCCGTCTCCATTGAGAGGCCTCAGACGCCAACGACAGTGCATGTGATGCACCGTGGAAGTCCGAGAGTGAGACCGGAACCGGACTTATATCAACCGGTGGGACACCCCAAGCGACAGAAAGGACTCAGCTATCACAGCTCCTATCTACGGATGTCATGCTCCTAAGACTGGGGGTCGCCAGTATTAAAATTTTCCAGCGTCCAGACCCAATTTTATCGCACCTATACTTAACTGCCCTCTTACGACTCATCTAATCACCAATCAGTCACGACCCCGCAAAATATTTCACCGGCTGTCGCCCGCTAAACCCGACCGCGGTGCAAGCCTCTCCCCTAGGAGCCTCGCCGTGTTGGCACGGTGGAATATTGCAACTGCACCTGGAGCACCGAAGACCATCAACCTGACACAGGCCCCTATGCTAATCACCCGCGGTATTAGGTATTGTGCAACCTGGGCAGGGGCTTAAGGGCTAGTCGGGTCTAATTGACCTGCTCGTTGATTGTGTTCCTCCGTGTCAGCCCGCTCTGTCATGGACAGAAAGGGAGTCGAAAAGGTGTGGTAGGGGGCTCAAGTGGCCTATAGACCAAGCCAAGTGACGTGTCGGGGTACCGTTCGAATTCGGCTGCCGCTCGTACCGGGTTTACCGGCCGATTGGATGACGCCCTCGCACTTTCCCGAAAACGGCTCGACTATCGGCCACCGGGACGAACAGGGGCTCAGGGATGAACTGCTTGGCCTTGCGGCTAATCCATAAGGGCACCTGCGGGAGCTACCCCCTTTCTTCAACATGGTCCGATGGTTCTGACGCACACATGCAGTCCTCAATCTCCGTCTCAACTGATACTACTTTCTCGGGTCATTGTCAGGAATCCCCGAACGCGCGTCCTATCATCCAGTATAATTAACAGCCCCTCCAAGTCACGCCAAGAAAACCGCGGCTAGGGGGGTATCATTAAGCACTGGCTGCCGGATCCCCGCTGGGTCTGACGGTTTAATGCTAGGCCCGCTTTTTGACATGTAGTGGTCAGCTACTAACTTCGGCGACTGGGGGCCCAAGAGACATGGGCCAAGCCTTGCTCCTGTACATCATGCTAGGACCGCACCTTGTCGTTTCCGTCCGAATGGCAACCTAACGTAAGAGAGCTGACACCGCCCCGGAGACAGGAAGGGCACTCTAACCACAAGCGGAATATTTGAGCACAGCAGCCCGCTGCGGCAGGCGTGGTAACCCAATGCGACGCCCGCAATGCCATAGCCAAAGAGCGCGTAATCCACCCCGCAGAATACCATGCCCGCCTGGCCAGGGCCACACTCACGCAGCTTACACAATACAATACATCCGGCAGAAGTTGGGCTTGACAGAAGAGGTCGTAGAAACGGCACAAACTCTCAAACGGGGATCCGAGGGCTAATTTAGGAGGGTTAGTAAAGGCTGGCACCACTTGAGTAGGCGGAGCTGTTTCCGAAGTCATCCCATAACGACCACACACCGACTGACCTGCTTCGGTCTACCTAAGCCCATCGATATACCGGGATATTTTACGCCGCCGCCCTCGTTATGTTACCCCTCAGTGTATACGTGTCTCTTCAGGTAGGCCCTTGATGGAAGAAGAACTACCAAGTCGTCGGTCCGAGGCGGGTGCGCACGAACGCAAAGAGCATGGCTTGGGCACTAAGCTTTAGGATTTGAGGGCGATAGATAAGTGACGCGGGCTCGGCAACGAAACGTTATCTGTCATAAGATTGGGCAGTGAGAGCCCCTCAAGCGGTAGCTCCCTCTAGCTTGCCGGGAGAATCCCACTATTTACCACCAGCGATACCCCCGTTGCGCGTTGTAGACCGGGGGAATGAAACAACCCTGCCACTCCCCCCGGGTCATATCCAGAGGAGTCCGTGTTCCGCGCGCCCGACAAGTGGCAGTATCCCTACATGTCGCGCCCTGAGCCTGGCAACCTCAACAGATGTGGTCTTCACGACCTCTACGCGATCATTACCGCCACCGGGTGAAACGGGATGAACGCGAGAACCGATTAAAAAAGTGGATGATAGTGACGGCAACATCTAATGGATACCGCTAGCAGTTTTTCTGCTTGCACCCTTACACGTTTCTGTGAAGCTTGGTTCCTGTGAGACGCTCGGACCCTGACACTCTTCATGGCCTCCAGTCATTCCACACCAGAGCCTGTGACATATGCCAGTGTTGCCCCTGCAGGCCTGCACCCGATCAGCCTGCCACTCGGCCCGGCCAGTAGGCAGGCTCATACCGGACCTCTATAGATTGCACTGCCTCGCCAGTAGGTGTTTCGTCACACGAGAGGTCCCTAGGAAGTACGATTTGAAGTCCGTATCCTCAGCCGCTCGAGCTACAAGCTCACCGTACATAGCTTCGGCTAGGCAATCATTCTGCCATATTAATCAGACCATGTGACGCCTATTGCCCCACTGTAATCCCATAGATCATCAGTGATCTCTAAGGGGCCCGGTCGTTCCCGCATTTGTGGATCGCTGCCGAGCCACTCTACCTGCATATACAGCCCCAACCCAATACACCACCATCACCAGCCATCCTTATACCCTGACTGCACCCAGTCCTAAGGTCAGAACCAGGAGAGCCTTGACAAGCCACTCATATTGAAAAGGGCACCGTTGTACCTAACGATACTCCTCTTTAGAGACAAGGGTAGCCGGCAGTCATCTCAACGATAGGTGCAGGTGATTTGTCAAAGCGCTCAGGAGCGAAGAGCCAACTATGAAACTCCACGACGTAAAACTACCCAGCTGGCAACTACTTGCTAGGTGTCTCGATAGGGCTCAGTGTGGGGTCGCTTGCCAAGGCTCAACGAGCAGCCTATGGGGTACCGGTGACATATCCGAACAATTGGCCGGTCACGCCTTGATAAAGTCTCAGTGGACTGTCTGCGTCCGCCAGTCCCCCGCAAAGCGTGTCGATTGGTTCTGGAATCAACAATTCCGCGCGGCTGTCAAGCTCGGCGCATGCCTTCTGGCACGAGAGCACTGGAGTTACTGGGGCCGGGTGGGGCTTCCATTACAAAGAAACCGGTCCTGCTTGACCATAGGGTTCGGTATAGGCTTAGCGAGTCCGTACGCCTATACGAAATTAAGAGGAGGTGGAGGTCCGATACGAAGTGGACCGATTCCCAACCGCCAAGGAGGCAGAGCTCTACGGGTCCCGCCGCTGTCCTGCAATGCACGTCGCCGTTTATTAGGATTGAGCCCCATCACAATCCTATCAATAACGCCTCTAAGCAGGGGGATGCGATATCTGGACTCACTGCCTTCCAGTTAGTTGCTAAGCGTCTCTAGGTCGTCTTAGGACATAGTCCCAGGAGTTATTTGGCGTTAGCGCGACGGCGCTTGAAGGCATGGTCAAGGCACGTTTTGATCAGTCATCGAGCCGCTTCTCGGCACGTTAGTCCCCAGGTATTCTGCTTGCTGTCCAAACAGTTCCAGATATCAGATAAGCCACACATCGAAGCGATTCCGACCAACTGCGCGAGACCAGCAAGCGCCGGCGCCTCTCCCGCATAGTTCCGTTGCATACGTCACGCACCGTCCCCGGTTTGTTCATGCTACACCTCGTCGATCTTCCCGCCTTGCCGTTAGACGCGCGACACATGTCTCGCTCATGACCACGACACGCGTCCGTTAGCCGGTCCTCGAAAGATACCCCCGAGTATAAAAGGGTTAATAACGTGGCTAACAGGCCCTGCGCTATTGGGCCAACACTACCATGTACTATCTACAGGCCACGGGCAGTCTTCATGCGGTCCAGCCCATTACGATCCTTAGGTTGGAAGCCTACCACAGTGCGCCACGGCTCGAGCCGGCTCGATCCACGCTTCACTAGTCCTTACGCAAGGTCGCAGCCTGGGGAGACGTCAGTTTGGCGCGACACGACTGCTATCATGTTCCACCATGCTTCATCACGCACCCTGCCCGCTATCGAGGAATCTCACTAACTCGCTCAGACCGTGTCAGGGTCCGGCACGATCGAACCGTAAAGCCGTCAATTAGGGTAGCTACGCATGCCCAACGCGGCTCTCCAACGTGAGAAGGCATCACTCTAGCGGCTCGAGAATGACGAGGCTTCTAGGTTCACAACTCTTAACGGATGCCACACTGGGCATGTAAGATGCTGCCTCCGCCTGCGGAAAACCAGAAACCTTTCGGTCTATCTGCCACTCACGACGCCAACCTATCCACGCTAAAGTACACAGTAGTCACTATTCGCCTTAACTTAGTTCAACAAGCGAGGTGTCGTCAAACGCAACCGGGTATCCTACGCGGGCCGCTATCCCGCCCTTACCACATGGCTAGGAACTGCTAGAACACCAAAGCGAACGGGCGACCCTACGTAGCCAGTACTAAATATCCACAATG

c) Use Genmark4, disponible en http://opal.biology.gatech.edu/GeneMark/, para buscar posibles
genes. Aplíquelo tanto al genoma original del plásmido como a la versión aleatorizada que generó.
Comente sobre las semejanzas o diferencias entre los resultados. (¿Hay más, hay menos? ¿Más
cortos, más largos?)

Esta pregunta fue respondida en el archivo PDF adjunto :3